In [107]:
import os
from pprint import pprint
from retrieve_iaa import retrieve_ann
from export_utils import MakeTable, ExportToLatex
from utils import converse_pgn, suffix_dict

from tf.app import use
from tf.fabric import Fabric

In [18]:
A = use(
    'bhsa', version='2017',
    mod=(
        'cmerwich/participant-analysis/coreference/tf:clone,'
        'etcbc/bh-reference-system/tf:clone'
    ), 
    hoist=globals(),
    silent=True)

In [1]:
from parse_ann import (TexFabricParse, EnrichMentions, 
                       IdentifyEntities, MakePandasTables, 
                       PrintThisTable, PrintSurvey, 
                       PrintPatternsAndNotes, PrintCorefID
                      )

In [3]:
my_book_name = 'Psalms'
from_chapter = 75
to_chapter = 75

In [209]:
mentions, corefs, suffix_errors, reconsider_rpt = ParseAnnotations(my_book_name, from_chapter, to_chapter)

yes man!


In [5]:
overall_df75, pos_df75, \
pronoun_df75, pronoun_pos_class_df75, \
pronoun_pos_sing_df75 = MakePandasTables(corefs75, mentions75)

In [6]:
PrintThisTable(overall_df75)

,total
mentions,68
singletons,17
classes,10


In [7]:
PrintThisTable(pos_df75)

,VP,NP,PPrP,suffix,DPrP,PrNP,total_type
in class,21,19,3,7,1,0,51
singleton,2,12,0,0,2,1,17
total,23,31,3,7,3,1,68
% total,34,46,4,10,4,1,100
first in chain,4,4,2,0,0,0,10
% chain,40,40,20,0,0,0,100


In [8]:
PrintThisTable(pronoun_df75)

,p1upl,p1usg,p2mpl,p2msg,p3fpl,p3fsg,p3mpl,p3msg,p3upl,umpl,total_pgn
in class,2,10,4,3,1,3,3,4,0,1,31
singleton,0,0,0,1,0,0,0,0,1,0,2
total,2,10,4,4,1,3,3,4,1,1,33
% total,6,30,12,12,3,9,9,12,3,3,100


In [9]:
PrintThisTable(pronoun_pos_class_df75)

,p1upl,p1usg,p2mpl,p2msg,p3fpl,p3fsg,p3mpl,p3msg,umpl,total_pgn
VP,2,7,4,0,1,0,2,4,1,21
suffix,0,0,0,3,0,3,1,0,0,7
PPrP,0,3,0,0,0,0,0,0,0,3
total,2,10,4,3,1,3,3,4,1,31
% total,6,32,13,10,3,10,10,13,3,100


In [10]:
#PrintSurvey(corefs75)
PrintPatternsAndNotes(corefs75, suffix_errors75)
#PrintCorefID(corefs)

C75:1 Who/what: HWDJNW / first: HWDJNW, type: VP
----------------------------------------------------------------------
HWDJNW -VP p1upl    HWDJNW -VP p1upl    

C75:5 Who/what: >LHJM / first: K, type: PPrP
----------------------------------------------------------------------
K -PPrP p2msg    >LHJM -NP    CM -NP    K -PPrP p2msg    NPL>WTJ -NP    K -PPrP p2msg    >QX -VP p1usg    >NJ -PPrP p1usg    >CPV -VP p1usg    >NKJ -PPrP p1usg    TKNTJ -VP p1usg    >MRTJ -VP p1usg    >LHJM -NP    CPV -NP    JCPJL -VP p3msg    JRJM -VP p3msg    JD&JHWH -NP    JMYW -VP p3mpl    >LHJ J<QB -NP    >GD< -VP p1usg    

C75:2 Who/what: >RY / first: NMGJM, type: VP
----------------------------------------------------------------------
NMGJM -VP umpl    >RY -NP    KL&JCBJ -NP    H -PPrP p3fsg    <MWDJ -NP    H -PPrP p3fsg    

C75:3 Who/what: HWLLJM / first: HWLLJM, type: NP
----------------------------------------------------------------------
HWLLJM -NP    THLW -VP p2mpl    

C75:8 Who/what: RC<JM / fir

In [11]:
mentions_all, corefs_all, suffix_errors_all = TexFabricParse('Psalms', 1, 150)
EnrichMentions(mentions_all)
IdentifyEntities(corefs_all)

In [182]:
def Pattern(i, lst, c, k, suffix_errors, mention_type, pgn_form):
    bad_words = [e[2] for e in suffix_errors]
    
    heading = f'C{k} Who: {c.id}, first: {c.first().surface}, type: {c.first().typ}, corpus class: {i}'
    
    if c.id != 'Singletons':
        if c.first().typ == mention_type and c.first().surface in pgn_form:
            i+=1
            print(heading, end='\n')
            lst.append(f'C{k}')
            pattern_list = []
            for m in c.terms:
                gloss = F.gloss.v(L.u(m.start, 'lex')[0])
                book, chapter, verse = T.sectionFromNode(m.start)
                
                if m.isSuffix and m.surface in bad_words:
                    print(verse, m.typ, m.surface, '!CORRUPT ANN', m.note, sep='\t', end='\n')
                    pattern_list.append(f'Sfx')
                
                elif m.isSuffix:
                    pgn_suffix = suffix_dict[m.surface][0]
                    print(verse, m.typ, pgn_suffix, m.surface, m.note, sep='\t', end='\n')
                    pattern_list.append(f'Sfx {pgn_suffix}')
                
                elif not m.isSuffix:
                    det = converse_pgn(F, m.start) if m.typ in {'VP', 'PPrP'} else ''
                    pattern_list.append(f'{m.typ} {det}')
                    print(verse, m.typ, det, gloss, m.surface, m.note, sep='\t', end='\n')
            print('Pattern: ', pattern_list)
            print('\n')
        
        elif (c.first().typ == mention_type and pgn_form == []) or \
            (c.first().surface in pgn_form and mention_type == ''):
            i+=1
            print(heading, end='\n')
            lst.append(f'C{k}')
            pattern_list = []
            for m in c.terms:
                book, chapter, verse = T.sectionFromNode(m.start)
                gloss = F.gloss.v(L.u(m.start, 'lex')[0])
                
                if m.isSuffix:
                    pgn_suffix = suffix_dict[m.surface][0]
                    print(verse, m.typ, pgn_suffix, m.surface, m.note, sep='\t', end='\n')
                    pattern_list.append(f'Sfx {pgn_suffix}')
                
                elif not m.isSuffix:
                    det = converse_pgn(F, m.start) if m.typ in {'VP', 'PPrP'} else ''
                    pattern_list.append(f'{m.typ} {det}')
                    print(verse, m.typ, det, gloss, m.surface, m.note, sep='\t', end='\n')
            print('Pattern: ', pattern_list)
            print('\n')

def RetrievePattern(cd, suffix_errors, mention_type, pgn):
    i = 0
    results_lst = []
    for k in cd:
        if k != 0:
            i+= 1
            Pattern(i, results_lst, cd[k], k, suffix_errors, mention_type, pgn)  
    print('Results: ', len(results_lst))
    return results_lst
    
RetrievePattern(corefs_all, suffix_errors_all, 'PPrP', ['NJ', 'J']) # 'PPrP', >NJ'

C3:5 Who: >QR>, first: J, type: PPrP, corpus class: 24
2	PPrP	p1usg	J	
2	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	PPrP	p3msg	W	
4	PPrP	p1usg	J	
4	PPrP	p1usg	J	
4	PPrP	p1usg	J	
5	PPrP	p1usg	J	
5	VP	p1usg	call	>QR>	
5	PPrP	p1usg	J	
6	PPrP	p1usg	i	>NJ	
6	VP	p1usg	lie down	CKBTJ	
6	VP	p1usg	sleep	>JCNH	
6	VP	p1usg	pass summer	HQJYWTJ	
6	PPrP	p1usg	NJ	
7	VP	p1usg	fear	>JR>	
7	PPrP	p1usg	J	
8	PPrP	p1usg	NJ	
8	PPrP	p1usg	J	
8	PPrP	p1usg	J	
Pattern:  ['Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p3msg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'PPrP p1usg', 'VP p1usg', 'VP p1usg', 'VP p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg']


C5:2 Who: >TPLL, first: J, type: PPrP, corpus class: 35
2	PPrP	p1usg	J	
2	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	VP	p1usg	pray	>TPLL	
4	PPrP	p1usg	J	
4	VP	p1usg	arrange	><RK	
4	VP	p1usg	look out	>YPH	
8	PPrP	p1usg	i	>NJ	
8	VP	p1usg	come	>BW>	
8	VP	p1usg	bow down	>CTXWH	
9	PPrP	

1	PPrP	p1usg	J	
1	VP	p1usg	fear	JR>	
1	PPrP	p1usg	J	
1	VP	p1usg	tremble	>PXD	
2	PPrP	p1usg	J	
2	PPrP	p1usg	J	
Pattern:  ['Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg']


C27:3 Who: <BD, first: J, type: PPrP, corpus class: 297
2	PPrP	p1usg	J	
2	PPrP	p1usg	J	
2	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	PPrP	p1usg	i	>NJ	
3	VP	umsg	trust	BWVX	
4	VP	p1usg	ask	C>LTJ	
4	VP	p1usg	seek	>BQC	
4	VP	uuu	sit	CBTJ	Infc with PreS.
4	PPrP	p1usg	J	
5	PPrP	p1usg	NJ	
5	PPrP	p1usg	NJ	
5	PPrP	p1usg	NJ	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
6	VP	p1usg	slaughter	>ZBXH	
6	VP	p1usg	sing	>CJRH	
6	VP	p1usg	sing	>ZMRH	
7	PPrP	p1usg	J	
7	VP	p1usg	call	>QR>	
7	PPrP	p1usg	NJ	
7	PPrP	p1usg	NJ	
8	PPrP	p1usg	J	
8	VP	p2mpl	seek	BQCW	Intuition: the heart (3Msg) addresses it's owner (2Msg) to seek YHWH's face.
8	VP	p1usg	seek	>BQC	
9	PPrP	p1usg	NJ	
9	NP		servant	<BD	
9	PPrP	p1usg	J	
9	PPrP	p1usg	NJ	
9	PPrP	p1usg	NJ	
9	PPrP	p1usg	J	
10	PPrP	p1usg	J	


4	VP	p1usg	trust	>BVX	
5	VP	p1usg	praise	>HLL	
5	VP	p1usg	trust	BVXTJ	
5	VP	p1usg	fear	>JR>	
5	PPrP	p1usg	J	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
7	PPrP	p1usg	J	
7	PPrP	p1usg	J	
9	PPrP	p1usg	J	
10	PPrP	p1usg	J	
10	VP	p1usg	call	>QR>	
10	VP	p1usg	know	JD<TJ	
10	PPrP	p1usg	J	
11	VP	p1usg	praise	>HLL	
11	VP	p1usg	praise	>HLL	
12	VP	p1usg	trust	BVXTJ	
12	VP	p1usg	fear	>JR>	
12	PPrP	p1usg	J	
13	PPrP	p1usg	J	
13	VP	p1usg	be complete	>CLM	
14	PPrP	p1usg	J	
14	PPrP	p1usg	J	
Pattern:  ['Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'PPrP p1usg', 'VP p1usg', 'VP p1usg', 'VP p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'VP p1usg', 'Sfx p1usg', 'VP p1usg', 'VP p1usg', 'VP p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg']


C57:2 Who: >XSH, first: NJ, type: PPrP, corpus class: 713
2	PPrP	p1usg	NJ	
2	PPrP	p1usg	NJ	
2	PPrP	p1usg	J	
2	VP	p1usg	seek refuge	>XSH	
3	VP	

12	PPrP	p1usg	J	
12	PPrP	p1usg	J	
12	PPrP	p1usg	J	
16	PPrP	p1usg	J	
Pattern:  ['Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg']


C94:11 Who: >MRTJ, first: J, type: PPrP, corpus class: 1276
16	PPrP	p1usg	J	Who is 1Csg?
16	PPrP	p1usg	J	
17	PPrP	p1usg	J	
17	PPrP	p1usg	J	
18	VP	p1usg	say	>MRTJ	
18	PPrP	p1usg	J	
18	PPrP	p1usg	NJ	
19	PPrP	p1usg	J	
19	PPrP	p1usg	J	
19	PPrP	p1usg	J	
22	PPrP	p1usg	J	
22	PPrP	p1usg	J	
22	PPrP	p1usg	J	
Pattern:  ['Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg']


C102:9 Who: >QR>, first: J, type: PPrP, corpus class: 1349
2	PPrP	p1usg	J	
2	PPrP	p1usg	J	
3	PPrP	p1usg	NJ	
3	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	VP	p1usg	call	>QR>	
3	PPrP	p1usg	NJ	
4	PPrP	p1usg	J	
4	PPrP	p1usg	J	
5	PPrP	p1usg	J	
5	VP	p1usg	forget	CKXTJ	
5	PPrP	p1usg	J	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
7	VP	p1

124	PPrP	p1usg	NJ	
125	NP		servant	<BD	
125	PPrP	p1usg	i	>NJ	
125	PPrP	p1usg	NJ	
125	VP	p1usg	know	>D<H	
127	VP	p1usg	love	>HBTJ	
128	VP	p1usg	be right	JCRTJ	
128	VP	p1usg	hate	FN>TJ	
129	PPrP	p1usg	J	
131	PPrP	p1usg	J	
131	VP	p1usg	open	P<RTJ	
131	VP	p1usg	gasp	>C>PH	
131	VP	p1usg	long	J>BTJ	
132	PPrP	p1usg	J	
132	PPrP	p1usg	NJ	
133	PPrP	p1usg	J	
133	PPrP	p1usg	J	
134	PPrP	p1usg	NJ	
134	VP	p1usg	keep	>CMRH	
135	NP		servant	<BD	
135	PPrP	p1usg	NJ	
136	PPrP	p1usg	J	
139	PPrP	p1usg	NJ	
139	PPrP	p1usg	J	
139	PPrP	p1usg	J	
140	NP		servant	<BD	
140	VP	p3msg	love	>HBH	
141	PPrP	p1usg	i	>NKJ	
141	VP	p1usg	forget	CKXTJ	
143	PPrP	p1usg	NJ	
143	PPrP	p1usg	J	
144	PPrP	p1usg	NJ	
144	VP	p1usg	be alive	>XJH	
145	VP	p1usg	call	QR>TJ	
145	PPrP	p1usg	NJ	
145	VP	p1usg	watch	>YRH	
146	VP	p1usg	call	QR>TJ	
146	PPrP	p1usg	NJ	
146	VP	p1usg	keep	>CMRH	
147	VP	p1usg	be in front	QDMTJ	
147	VP	p1usg	cry	>CW<H	
147	VP	p1usg	wait, to hope	JXLTJ	
148	PPrP	p1usg	J	
148	VP	uuu	be concerned with	FJX	
149	PPrP	p1usg	J

Pattern:  ['Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg']


C142:2 Who: >Z<Q, first: J, type: PPrP, corpus class: 1896
2	PPrP	p1usg	J	
2	VP	p1usg	cry	>Z<Q	
2	PPrP	p1usg	J	
2	VP	p1usg	favour	>TXNN	
3	VP	p1usg	pour	>CPK	
3	PPrP	p1usg	J	
3	PPrP	p1usg	J	
3	VP	p1usg	report	>GJD	
4	PPrP	p1usg	J	
4	PPrP	p1usg	J	
4	PPrP	p1usg	J	
4	VP	p1usg	walk	>HLK	
4	PPrP	p1usg	J	
5	PPrP	p1usg	J	
5	PPrP	p1usg	NJ	
5	PPrP	p1usg	J	
6	VP	p1usg	cry	Z<QTJ	
6	VP	p1usg	say	>MRTJ	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
7	PPrP	p1usg	J	
7	VP	p1usg	belittle	DLWTJ	
7	PPrP	p1usg	NJ	
7	PPrP	p1usg	J	
7	PPrP	p1usg	NJ	
8	PPrP	p1usg	J	
8	PPrP	p1usg	J	
8	PPrP	p1usg	J	
Pattern:  ['Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'VP p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'VP p1usg', 'S

In [ ]:
def Search(i, lst, c, k, suffix_errors, what):
    pass




def FindMention(cd, suffix_errors, what):
    i = 0
    results_lst = []
    for k in cd:
        if k != 0:
            i+= 1
            Search(i, results_lst, cd[k], k, suffix_errors, what)
        Search(i, results_lst, cd[k], k, suffix_errors, what)
        
    print('Results: ', len(results_lst))
    return results_lst
FindMention(corefs_all, suffix_errors_all, what)

In [203]:
def GetPatterns(i, lst, c, k, suffix_errors, who):
    bad_words = [e[2] for e in suffix_errors]
    if c.id != 'Singletons':
        if c.id in who:# and c.first().surface == pgn:
            i+=1
            print(f'C{k} Who: {c.id}, first: {c.first().surface}, type: {c.first().typ}, corpus class: {i}', 
                  end='\n')
            lst.append(f'C{k}')
            pattern_list = []
            for m in c.terms:
                gloss = F.gloss.v(L.u(m.start, 'lex')[0])
                book, chapter, verse = T.sectionFromNode(m.start)
                
                if m.typ in {'VP', 'PPrP'} and not m.isSuffix:
                    pgn = converse_pgn(F, m.start)
                    pattern_list.append(f'{m.typ} {pgn}')
                    print(verse, m.typ, pgn, gloss, m.surface, m.note, sep='\t', end='\n')
                
                elif m.isSuffix and m.surface in bad_words:
                    print(verse, m.typ, m.surface, '!CORRUPT ANN', m.note, sep='\t', end='\n')
                    pattern_list.append(f'Sfx')
                
                elif m.isSuffix:
                    pgn_suffix = suffix_dict[m.surface][0]
                    print(verse, m.typ, pgn_suffix, m.surface, m.note, sep='\t', end='\n')
                    pattern_list.append(f'Sfx {pgn_suffix}')
                
                else:
                    print(verse, m.typ, gloss, m.surface, m.note, sep='\t', end='\n')
                    pattern_list.append(m.typ)
            print('Pattern: ', pattern_list)
            print('\n')
            
def FindWho(cd, suffix_errors, who):
    i = 0
    results_lst = []
    for k in cd:
        if k != 0:
            i+= 1
            GetPatterns(i, results_lst, cd[k], k, suffix_errors, who)
    print('Results: ', len(results_lst))
    return results_lst

FindWho(corefs_all, suffix_errors_all, ['XSJTJ']) #['>LHJM', '>LHJ', '>L', 'JHWH'] #PPrP p2msg
#FindWho(corefs_all, suffix_errors_all, ['>L'])

C7:3 Who: XSJTJ, first: J, type: PPrP, corpus class: 58
2	PPrP	p1usg	J	
2	VP	p1usg	seek refuge	XSJTJ	
2	PPrP	p1usg	NJ	
2	PPrP	p1usg	J	
2	PPrP	p1usg	NJ	
3	PPrP	p1usg	J	
4	PPrP	p1usg	J	
4	VP	p1usg	make	<FJTJ	
4	PPrP	p1usg	J	
5	VP	p1usg	deal fully	GMLTJ	
5	PPrP	p1usg	J	
5	VP	p1usg	draw off	>XLYH	
5	PPrP	p1usg	J	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
6	PPrP	p1usg	J	
7	PPrP	p1usg	J	
7	PPrP	p1usg	J	
9	PPrP	p1usg	NJ	
9	PPrP	p1usg	J	
9	PPrP	p1usg	J	
9	PPrP	p1usg	J	
11	PPrP	p1usg	J	
18	VP	p1usg	praise	>WDH	
18	VP	p1usg	sing	>ZMR	
Pattern:  ['Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'VP p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'Sfx p1usg', 'VP p1usg', 'VP p1usg']


C11:1 Who: XSJTJ, first: XSJTJ, type: VP, corpus class: 110
1	VP	p1usg	seek refuge	XSJTJ	
1	PPrP	p1usg	J	
1	VP	p2fsg	waver	NWDW	
1	PPrP	p2mpl	KM	
P

['C7:3', 'C11:1', 'C16:4', 'C31:6', 'C71:3', 'C144:1']

## IAA: Manual Annotations and MiMi

In [188]:
path_manual = os.path.expanduser('~/Sites/brat/data/coref/Psalms/annotate/Psalms_075.ann')
path_mimi = os.path.expanduser('~/github/cmerwich/participant-analysis/iaa-ann-vs-mimi/Psalms_075.ann')

In [189]:
ps075_df = MakeTable('~/github/cmerwich/participant-analysis/iaa-ann-vs-mimi/Psalms_075.iaa')
ps075_df

,ann_A,ann_B,L,M,R,D,d
0,C1,C2,0,2,0,0,0.0000
1,C2,C5,5,1,1,6,0.8571
2,C3,C7,2,0,2,4,1.0000
3,C4,C6,2,2,1,3,0.6000
4,C5,C4,16,4,0,16,0.8000
5,C6,C8,1,1,1,2,0.6667
6,C7,C1,5,0,4,9,1.0000
7,C8,C3,5,0,2,7,1.0000
8,C9,C9,0,3,1,1,0.2500
9,C10,C10,0,2,0,0,0.0000


In [190]:
retrieve_ann(path_manual, path_mimi)

ann_A	ann_B	L	M	R	D	d	

------------------------------------------------------------
C1	C2	0	2	0	0	0.0
------------------------------------------------------------
A	['HWDJNW', 'HWDJNW']	

B	['HWDJNW', 'HWDJNW']	

------------------------------------------------------------
C2	C5	5	1	1	6	0.8571
------------------------------------------------------------
A	['NMGJM', '>RY', 'KL&JCBJ', 'H', '<MWDJ', 'H']	

B	['NMGJM', '>RY W-KL JCBJ']	

A diff	[('>RY', 1), ('KL&JCBJ', 2), ('H', 3), ('<MWDJ', 4), ('H', 5)]	

B diff	[('>RY W-KL JCBJ', 1)]	

------------------------------------------------------------
C3	C7	2	0	2	4	1.0
------------------------------------------------------------
A	['HWLLJM', 'THLW']	

B	['JCPJL', 'JRJM']	

A diff	[('HWLLJM', 0), ('THLW', 1)]	

B diff	[('JCPJL', 0), ('JRJM', 1)]	

------------------------------------------------------------
C4	C6	2	2	1	3	0.6
------------------------------------------------------------
A	['TRJMW', 'QRNK', 'M', 'TDBRW']	

B	['TRJMW', 'TRJMW', 